<a href="https://colab.research.google.com/github/lovestar7914/study_data_analytics/blob/main/codes/NLP/healthapp_review_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## refer : https://heytech.tistory.com/401

In [12]:
 !pip install python-mecab-ko pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 65.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 82.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 39.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.

In [1]:
import pandas as pd
df_raw = pd.read_excel('./dataset_raw.xlsx')
df_raw[:2]

,app,review,rating
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1
1,다리 근육 운동 – 4주 프로그램,굿,5


## 전처리

### 결측치 처리

In [2]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 540076 entries, 0 to 540075
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   app     540076 non-null  object
 1   review  540074 non-null  object
 2   rating  540076 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 12.4+ MB


In [3]:
df_raw_dropna = df_raw.dropna()
df_raw_dropna.isnull().sum()

app       0
review    0
rating    0
dtype: int64

### 자연어 전처리

#### 특정(상업용) 앱 리스트에서 제거

In [4]:
remove_app_list = pd.read_excel('./remove_app_list.xlsx')
remove_app_list[:5]

,app
0,캐시슬라이드 스텝업 - 걸음에 포인트를 더하다
1,만보기 - 걸음 계산기
2,딱 1주일 다이어트 습관 : 요요없는 건강한 다이어트
3,타임캐시 – 돈버는 어플
4,돈버는어플 - 캐시런


In [5]:
remove_app_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106 entries, 0 to 105
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   app     106 non-null    object
dtypes: object(1)
memory usage: 976.0+ bytes


In [6]:
for remove_app in remove_app_list['app']:
  try:
    df_removed_app = df_raw_dropna[df_raw_dropna['app'] != remove_app]
  except :
    pass

In [7]:
# import re
# review_removed = list(map(lambda review: re.sub('[^가-힣 ]', '', review), df_removed_app['review']))
# df_removed_app['review'] = review_removed

In [8]:
# df_removed_app = df_removed_app[500:9000]

### 형태소 적용과 불용어 처리

In [9]:
# 불용어 사전 불러오기
df_stopwords = pd.read_excel('./stopword_list.xlsx')
df_stopwords[:5]

,stopword
0,가까스로
1,가량
2,가령
3,가민
4,가민커넥트


In [10]:
from mecab import MeCab
mecab = MeCab()
stopwords = df_stopwords['stopword'].values
# 토크나이징 함수 정의(한자 이상 적용, 불용어 삭제)
def tokenizer(raw, stopword=stopwords):
    result_list = list()
    try:
      for word in mecab.nouns(raw) :
          if len(word) > 1 and word not in stopword :
              result_list.append(word)
      # 일부 경우(토큰 갯수 제한)
      # if len(result_list) <= 3 or len(result_list) < 12 :
      #   result_list
    except :
      pass

    return ' '.join(result_list) # return str

In [11]:
# mecab.nouns('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.')

In [12]:
# 형태 분석기 적용된 함수 실행
tokenizer('아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.')

'운동'

In [13]:
df_removed_app['review'].apply(tokenizer)

0                        연동 기존
1                             
2                           최고
3                           운동
4                             
                  ...         
540071                      단다
540072                        
540073                      필요
540074                  포인트 최고
540075    최고 가능 필요 감사 회원 가입 필요
Name: review, Length: 540074, dtype: object

In [14]:
df_removed_app['tokenized_review'] = df_removed_app['review'].apply(tokenizer)
df_removed_app[:2]

,app,review,rating,tokenized_review
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1,연동 기존
1,다리 근육 운동 – 4주 프로그램,굿,5,


#### 유사어 대치

In [15]:
df_replace_list = pd.read_excel('./replace_list.xlsx')
df_replace_list[:3]

,before_replacement,after_replacement
0,S헬스,삼성헬스
1,LG폰,스마트폰
2,LG V10,스마트폰


In [16]:
def replace_word(tokenized_review):
  replaced_review = str()
  for i in range(len(df_replace_list['before_replacement'])):
    try:
      # 치환할 단어가 있는 경우에만 데이터 치환 수행
      if df_replace_list['before_replacement'][i] in tokenized_review:
        replaced_review = tokenized_review.replace(df_replace_list['before_replacement'][i]
                                                   , df_replace_list['after_replacement'][i])
      else :
        replaced_review = tokenized_review
    except Exception as e:
      print(f"Error 발생 / 에러명: {e}")
      # replaced_review = tokenized_review

  return replaced_review

In [17]:
replace_word('S헬스 한글 LG V10 업데이트 완벽')

'S헬스 한글 LG V10 업데이트 완벽'

In [18]:
replace_word('한글 업데이트 완벽')

'한글 업데이트 완벽'

In [19]:
replace_word('')

''

In [20]:
df_removed_app['replaced_review'] = df_removed_app['tokenized_review'].apply(replace_word)
df_removed_app[:2]

,app,review,rating,tokenized_review,replaced_review
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1,연동 기존,연동 기존
1,다리 근육 운동 – 4주 프로그램,굿,5,,


In [21]:
df_removed_app.to_csv('./preprocess_reviews.csv')

### 부정과 긍정 댓글 분리

In [22]:
condition_positive = df_removed_app['rating'] >= 3  # 긍정
condition_negative = df_removed_app['rating'] < 3  # 부정

In [23]:
df_review_positive = df_removed_app[condition_positive]
df_review_negative = df_removed_app[condition_negative]

In [24]:
df_review_positive[:3]

,app,review,rating,tokenized_review,replaced_review
1,다리 근육 운동 – 4주 프로그램,굿,5,,
2,다리 근육 운동 – 4주 프로그램,최고입니디,5,최고,최고
3,다리 근육 운동 – 4주 프로그램,아무곳에서나 보고 억지로라도 운동할수 있어서 너무 좋습니다. 감사히 잘 쓸께요.😂,5,운동,운동


In [25]:
df_review_negative[:3]

,app,review,rating,tokenized_review,replaced_review
0,다리 근육 운동 – 4주 프로그램,다른 P4P어플과 연동 하면 기존에 있던 스케쥴이 싹 사라짐,1,연동 기존,연동 기존
18,Notify & Fitness for Mi Band,다른 것보다 문자수신시 글자가 깨져서 안 보여요. 프로 구입했는데 환불해 주세요,2,문자 수신 글자 구입 환불,문자 수신 글자 구입 환불
20,Notify & Fitness for Mi Band,pro결제 했는데 연결이 안됩니다 환불해주세요!,1,결제 연결 환불,결제 연결 환불


In [26]:
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51367 entries, 0 to 540058
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   app               51367 non-null  object
 1   review            51367 non-null  object
 2   rating            51367 non-null  int64 
 3   tokenized_review  51367 non-null  object
 4   replaced_review   51367 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


In [27]:
df_review_negative.dropna(inplace=True)
df_review_positive = df_review_positive.dropna()

<ipython-input-27-b8e682f71e14>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_review_negative.dropna(inplace=True)


In [28]:
df_review_negative.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51367 entries, 0 to 540058
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   app               51367 non-null  object
 1   review            51367 non-null  object
 2   rating            51367 non-null  int64 
 3   tokenized_review  51367 non-null  object
 4   replaced_review   51367 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


### 긍정, 부정으로 전처리한 것을 CSV 파일로 저

In [29]:
df_review_positive.to_csv('./review_positive.csv')
df_review_negative.to_csv('./review_negative.csv')